<a href="https://colab.research.google.com/github/peppertaco/AI/blob/main/TavernOAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##태번 실행기

In [ ]:
#@title ▼모바일 사용 시 미디어를 재생시켜 놓으세요 { display-mode: "form" }

%%html
<b>Press play on the music player to keep the tab alive, then start TavernAI below (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title TavernAI
#@markdown **<font color = red><--눌러 (≖ ‸ ≖ ✿)**
#@markdown ###### ***옵션. NGROK 사용 ( 입력시 자동실행 )***
#@markdown - loca.lt 링크보다 ngrok 로딩이 더 빠름 
#@markdown - https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_token = "" #@param {type:"string"}
Model = "Tavern-scale-v1" #@param ["Tavern-scale-v1","Tavern-scale-v2-swipe","1.3.1 (public)", "v0.2 (GPT3.5 only)"] {allow-input: false}
use_google_drive = True #@param {type:"boolean"}
#@markdown - <font color = red> 공식 포함, 모든 버전 GPT3.5 무한 리퀘스트 버그 있습니다.</font>
#@markdown - <font color = red> GPT3.5 를 사용하려면 모든 모델에서 OpenAI API를 선택하면 됩니다.</font>
#@markdown - <font color = red> v0.2 (GPT3.5 only) 토크나이저 미적용으로 토큰 손실있습니다.</font>
#@markdown ---
#@markdown ---
#@markdown - 무료 급식소
replace = "" #@param {type:"string"}

#@markdown ---
#@markdown ---
##@markdown - <font color = red> Silly Lossy 전용 옵션</font>
#extras_enable_captioning = True #@param {type:"boolean"}
#Captions_Model = "Salesforce/blip-image-captioning-large" #@param [ "Salesforce/blip-image-captioning-large", "Salesforce/blip-image-captioning-base" ]
##@markdown * Salesforce/blip-image-captioning-large - good base model
##@markdown * Salesforce/blip-image-captioning-base - slightly faster but less accurate
##extras_enable_emotions = True #@param {type:"boolean"}
#Emotions_Model = "joeddav/distilbert-base-uncased-go-emotions-student" #@param ["bhadresh-savani/distilbert-base-uncased-emotion", "joeddav/distilbert-base-uncased-go-emotions-student"]
##@markdown * bhadresh-savani/distilbert-base-uncased-emotion = 6 supported emotions<br>
##@markdown * joeddav/distilbert-base-uncased-go-emotions-student = 28 supported emotions
#extras_enable_memory = True #@param {type:"boolean"}
#Memory_Model = "Qiliang/bart-large-cnn-samsum-ChatGPT_v3" #@param [ "Qiliang/bart-large-cnn-samsum-ChatGPT_v3", "Qiliang/bart-large-cnn-samsum-ElectrifAi_v10", "distilbart-xsum-12-3" ]
##@markdown * Qiliang/bart-large-cnn-samsum-ChatGPT_v3 - summarization model optimized for chats
##@markdown * Qiliang/bart-large-cnn-samsum-ElectrifAi_v10 - nice results so far, but still being evaluated
##@markdown * distilbart-xsum-12-3 - faster, but pretty basic alternative

!nvidia-smi
!npm install -g localtunnel
import subprocess
import time
import sys
import os
import threading
import shutil
from google.colab import drive

if NGROK_token:
  !pip install pyngrok
  from pyngrok import ngrok
  os.environ['NGROK_token'] = NGROK_token
  !ngrok authtoken $NGROK_token
  !streamlit run app.py&>/dev/null&
  publ_url = ngrok.connect(addr = '8000')
else:
  publ_url = ''
  print("NGROK 미사용")
    
    
if use_google_drive:
  drive.mount('/content/drive/')
  if not os.path.exists("/content/drive/MyDrive/TavernAI/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/characters/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/characters/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/chats/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/chats/")
else:
  if not os.path.exists("/content/drive"):
    os.mkdir("/content/drive")
  if not os.path.exists("/content/drive/MyDrive/"):
    os.mkdir("/content/drive/MyDrive/")

def copy_characters(use_google_drive=False):
  if not use_google_drive:
    return
  
  src_folder = "/TavernAIColab/public/characters"
  dst_folder = "/content/drive/MyDrive/TavernAI/characters"

  for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)

    if os.path.exists(dst_file):
      print(f"{dst_file} already exists. Skipping...")
      continue

    shutil.copy(src_file, dst_folder)
    print(f"{src_file} copied to {dst_folder}")

extras_url = ''
if Model in ["Silly Waifu v0.16"]:
  params = []

  params.append('--cpu')

  ExtrasModules = []

  if (extras_enable_captioning):
   ExtrasModules.append('caption')

  if (extras_enable_memory):
    ExtrasModules.append('summarize')

  if (extras_enable_emotions):
    ExtrasModules.append('classify')

  params.append(f'--classification-model={Emotions_Model}')
  params.append(f'--summarization-model={Memory_Model}')
  params.append(f'--captioning-model={Captions_Model}')
  params.append(f'--enable-modules={",".join(ExtrasModules)}')

#TavernAI extras
  %cd /
  !git clone https://github.com/Cohee1207/TavernAI-extras
  %cd TavernAI-extras
  !pip install -r requirements.txt
  !pip install tensorflow==2.11
  cmd = f"python server.py {' '.join(params)}"
  print(cmd)

  extras_process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd='/TavernAI-extras', shell=True)
  print('processId:', extras_process.pid)

  while True:
      line = extras_process.stdout.readline().decode().strip()
      if "Running on " in line:
         break
      if not line:
         print('breaking on line')
         break
      print(line)

  subprocess.call('nohup lt --port 5100 > ./extras.out 2> ./extras.err &', shell=True)
  print('Waiting for lt init...')
  time.sleep(5)

  while True:
    if (os.path.getsize('./extras.out') > 0):
     with open('./extras.out', 'r') as f:
       lines = f.readlines()
       for x in range(len(lines)):
         if ('your url is: ' in lines[x]):
           print('TavernAI Extensions URL:')
           extras_url = lines[x].split('your url is: ')[1]
           print(extras_url)
       break
    if (os.path.getsize('./extras.err') > 0):
     with open('./extras.err', 'r') as f:
       print(f.readlines())
       break

#TavernAI
%cd /
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.37.2/install.sh | bash
!nvm install 19.1.0
!nvm use 19.1.0
!node -v

if Model == "1.3.1 (public)":
  !git clone https://github.com/peppertaco/TavernAI TavernAIColab
elif Model == "v0.2 (GPT3.5 only)":
  !git clone https://github.com/peppertaco/TavernAIColab
elif Model == "Tavern-scale-v1":
  !git clone -b new4 https://github.com/peppertaco/TavernAIColab  
elif Model == "Tavern-scale-v2-swipe":
  !git clone -b new4-swipe https://github.com/peppertaco/TavernAIColab  
elif Model == "Silly Waifu v0.16":
  !git clone -b waifu-colab https://github.com/peppertaco/Tavern TavernAIColab 
  
copy_characters(use_google_drive)

api_openai = "https://api.openai.com/v1"
file_path = "/TavernAIColab/server.js"

if replace:
    Proxy = replace
else:
    Proxy = ""
with open(file_path, "r") as file:
    content = file.read()
content = content.replace(api_openai, replace)
if replace:
    Proxy = replace
else:
    Proxy = ""
with open(file_path, "r") as file:
    content = file.read()
if replace:
    content = content.replace(api_openai, replace)
else:
    content = content
if Proxy:
    content = content.replace(replace, Proxy)
with open(file_path, "w") as file:
    file.write(content)

%cd TavernAIColab
!npm install
time.sleep(1)
%env colab=2
%env colaburl=$url
if use_google_drive:
  %env googledrive=2
!nohup node server.js &
time.sleep(3)
#print('### TavernAI Extensions LINK ###')
#print(extras_url)
#print('')
print('###TavernAI LINK###')
print(publ_url)
!lt --port 8000

In [ ]:
#@title 공식 Silly 태번 전용  (원본 https://github.com/Cohee1207/SillyTavern)
#@markdown **<font color = red><--눌러 (≖ ‸ ≖ ✿)**
#@markdown ###### ***옵션. NGROK 사용 ( 입력시 자동실행 )***
#@markdown - loca.lt 링크보다 ngrok 로딩이 더 빠름 
#@markdown - https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_token = "" #@param {type:"string"}
#@markdown ---
#@markdown ---
ForceInitSteps = []
UseGoogleDrive = True #@param {type:"boolean"}
ModelsFromDrive = False
UseExtrasExtensions = True #@param {type:"boolean"}
#@markdown Enables hosting of extensions backend for TavernAI Extras
extras_enable_captioning = True #@param {type:"boolean"}
#@markdown Loads the image captioning module
Captions_Model = "Salesforce/blip-image-captioning-large" #@param [ "Salesforce/blip-image-captioning-large", "Salesforce/blip-image-captioning-base" ]
#@markdown * Salesforce/blip-image-captioning-large - good base model
#@markdown * Salesforce/blip-image-captioning-base - slightly faster but less accurate
extras_enable_emotions = True #@param {type:"boolean"}
#@markdown Loads the sentiment classification model
Emotions_Model = "joeddav/distilbert-base-uncased-go-emotions-student" #@param ["bhadresh-savani/distilbert-base-uncased-emotion", "joeddav/distilbert-base-uncased-go-emotions-student"]
#@markdown * bhadresh-savani/distilbert-base-uncased-emotion = 6 supported emotions<br>
#@markdown * joeddav/distilbert-base-uncased-go-emotions-student = 28 supported emotions
extras_enable_memory = True #@param {type:"boolean"}
#@markdown Loads the story summarization module
Memory_Model = "Qiliang/bart-large-cnn-samsum-ChatGPT_v3" #@param [ "Qiliang/bart-large-cnn-samsum-ChatGPT_v3", "Qiliang/bart-large-cnn-samsum-ElectrifAi_v10", "distilbart-xsum-12-3" ]
#@markdown * Qiliang/bart-large-cnn-samsum-ChatGPT_v3 - summarization model optimized for chats
#@markdown * Qiliang/bart-large-cnn-samsum-ElectrifAi_v10 - nice results so far, but still being evaluated
#@markdown * distilbart-xsum-12-3 - faster, but pretty basic alternative

%cd /content

!cat .ii
!nvidia-smi

import os, subprocess, time, pathlib, json, base64, sys
if NGROK_token:
  !pip install pyngrok
  from pyngrok import ngrok
  os.environ['NGROK_token'] = NGROK_token
  !ngrok authtoken $NGROK_token
  !streamlit run app.py&>/dev/null&
  publ_url = ngrok.connect(addr = '5001')
else:
  publ_url = ''
  print("NGROK 미사용")

# ---
# Utils
class IncrementialInstall:
  def __init__(self, root = "/", tasks = [], force = []):
    self.tasks = tasks
    self.path = os.path.join(root, ".ii")
    self.completed = list(filter(lambda x: not x in force, self.__completed()))

  def __completed(self):
    try:
      with open(self.path) as f:
        return json.load(f)
    except:
      return []

  def addTask(self, name, func):
    self.tasks.append({"name": name, "func": func})

  def run(self):
    todo = list(filter(lambda x: not x["name"] in self.completed, self.tasks))
    try:
      for task in todo:
        task["func"]()
        self.completed.append(task["name"])
    finally:
      with open(self.path, "w") as f:
        json.dump(self.completed, f)

def create_paths(paths):
  for directory in paths:
    if not os.path.exists(directory):
      os.makedirs(directory)

def link(srcDir, destDir, files):
  '''
    Link source to dest copying dest to source if not present first
  '''
  for file in files:
    source = os.path.join(srcDir, file)
    dest = os.path.join(destDir, file)
    if not os.path.exists(source):
      !cp -r "$dest" "$source"
    !rm -rf "$dest"
    !ln -fs "$source" "$dest"

from google.colab import drive
if UseGoogleDrive:
  drive.mount("/content/drive/")
else:
  create_paths([
      "/content/drive/MyDrive"
  ])

ii = IncrementialInstall(force=ForceInitSteps)

# ---
# SillyTavern py modules
def cloneTavern():
  %cd /
  !git clone https://github.com/Cohee1207/SillyTavern
  %cd /SillyTavern
  !git reset --hard 50526a16b94a06ebc4b4966da1b59a0fd9ea546f
  %cd -
  !cp /SillyTavern/colab/*.py ./
ii.addTask("Clone SillyTavern", cloneTavern)
ii.run()

# ---
# nodejs
%cd /
def setupNVM():
  !curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.37.2/install.sh | bash
ii.addTask("Setup NVM", setupNVM)

def installNode():
  !nvm install 19.1.0
  !nvm use 19.1.0
ii.addTask("Install node", installNode)


# ---
# TavernAI extras
import globals
globals.extras_url = '(disabled)'
globals.params = []
globals.params.append('--cpu')
ExtrasModules = []

if (extras_enable_captioning):
  ExtrasModules.append('caption')
if (extras_enable_memory):
  ExtrasModules.append('summarize')
if (extras_enable_emotions):
  ExtrasModules.append('classify')

globals.params.append(f'--classification-model={Emotions_Model}')
globals.params.append(f'--summarization-model={Memory_Model}')
globals.params.append(f'--captioning-model={Captions_Model}')
globals.params.append(f'--enable-modules={",".join(ExtrasModules)}')


if UseExtrasExtensions:
    def cloneExtras():
        %cd /
        !git clone https://github.com/Cohee1207/TavernAI-extras
    ii.addTask('clone extras', cloneExtras)

    def installRequirements():
        %cd /TavernAI-extras
        !npm install -g localtunnel
        !pip install -r requirements.txt
        !pip install tensorflow==2.11
    ii.addTask('install requirements', installRequirements)

    from extras_server import runServer, extractUrl
    ii.addTask('run server', runServer)
    ii.addTask('extract extras URL', extractUrl)

%cd /SillyTavern

if UseGoogleDrive:
  %env googledrive=2

  def setupTavernPaths():
    %cd /SillyTavern
    tdrive = "/content/drive/MyDrive/SillyTavern"
    create_paths([
        tdrive,
        os.path.join("public", "groups"),
        os.path.join("public", "group chats")
    ])
    link(tdrive, "public", [
        "settings.json",
        "backgrounds",
        "characters",
        "chats",
        "User Avatars",
        "worlds",
        "group chats",
        "groups",
    ])
  ii.addTask("Setup Tavern Paths", setupTavernPaths)

def installTavernDependencies():
  %cd /SillyTavern
  !npm install
  !npm install -g localtunnel
ii.addTask("Install Tavern Dependencies", installTavernDependencies)
ii.run()

%env colaburl=$url
%env SILLY_TAVERN_PORT=5001
print('###Extensions API LINK###', globals.extras_url,  "###SillyTavern LINK###", sep="\n")
print(publ_url)
p = subprocess.Popen(["lt", "--port", "5001"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(p.stdout.readline().decode().strip())
!node server.js

In [ ]:
#@title Silly 확장기능 전용 (임시)
#@markdown - **출력창에** *Running on http://localhost:5100* **까지 뜨는거 확인하고 사용하세요**
#@markdown - https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_token = "" #@param {type:"string"}
import os, subprocess, time, pathlib, json, base64, sys
if NGROK_token:
  !pip install pyngrok
  from pyngrok import ngrok
  os.environ['NGROK_token'] = NGROK_token
  !ngrok authtoken $NGROK_token
  !streamlit run app.py&>/dev/null&
  publ_url = ngrok.connect(addr = '5100')
else:
  publ_url = ''
  print("NGROK 미사용")
%cd /
!git clone https://github.com/Cohee1207/TavernAI-extras
%cd TavernAI-extras
!pip install -r requirements.txt
!pip install tensorflow==2.11
cmd = f"python server.py --enable-modules=caption,classify,summarize --classification-model=joeddav/distilbert-base-uncased-go-emotions-student --summarization-model=Qiliang/bart-large-cnn-samsum-ChatGPT_v3 --captioning-model=Salesforce/blip-image-captioning-large --cpu"
p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print('Use this url :',publ_url)   
for line in iter(p.stdout.readline, b''):
    print(line.decode().strip())
!lt --port 5100